In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

In [35]:
# Create dataframe from cities collected with weather api, and turned into csv file
cities_df = pd.read_csv("~/desktop/creativity/data_analytics/git/weather_api/data/weather_data.csv")
cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Windspeed (mph),Cloudiness (%)
0,0,Thinadhoo,MV,0.5333,72.9333,83.73,72,16.44,100
1,1,Torbay,CA,47.6666,-52.7314,34.54,64,13.80,90
2,2,Qaanaaq,GL,77.4840,-69.3632,31.44,75,6.71,5
3,3,Arraial do Cabo,BR,-22.9661,-42.0278,73.26,60,5.75,75
4,4,Deputatsky,RU,69.3000,139.9000,42.40,59,4.27,14


In [36]:
gmaps.configure(g_key)

In [37]:
# Create heatmap for location of cities, and level of humidity
# Set location of cities
locations = cities_df[["Latitude", "Longitude"]]
# Set humidity of cities, which will be used as the weight for heat layer
humidity = cities_df["Humidity (%)"]
fig = gmaps.figure()
# Build heat layer
heatLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
# Add heat layer to map
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Narrow data to ideal weather conditions
# Temp 70 degrees > 80 degrees
# Windspeed < 10mph
# Cloudiness 0%
idealCities_df = cities_df.loc[(cities_df["Temperature (F)"] < 80) & (cities_df["Temperature (F)"] > 70) \
                                    & (cities_df["Windspeed (mph)"] < 10) \
                                    & (cities_df["Cloudiness (%)"] == 0)].dropna()
idealCities_df

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Windspeed (mph),Cloudiness (%)
75,75,Jalu,LY,29.0331,21.5482,79.54,12,7.49,0
225,225,Antalya Province,TR,36.7741,30.7178,77.59,44,5.61,0
412,412,Port Said,EG,31.2565,32.2841,73.38,73,6.91,0
462,462,Sweetwater,US,25.7634,-80.3731,79.21,59,1.99,0
528,528,Athens,GR,37.9795,23.7162,75.24,44,4.00,0


In [44]:
# Create new dataframe using narrowed down cities, with columns for the city, country, latitude, and longitude
hotel_df = idealCities_df[["City", "Country", "Latitude", "Longitude"]].copy()
# Add a new column for the name of the first hotel within 5000m of city
hotel_df["Hotel"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel
75,Jalu,LY,29.0331,21.5482,
225,Antalya Province,TR,36.7741,30.7178,
412,Port Said,EG,31.2565,32.2841,
462,Sweetwater,US,25.7634,-80.3731,
528,Athens,GR,37.9795,23.7162,


In [58]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set criteria to find hotel
# 5000m of city
# hotels
# Google api key
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

In [68]:
#Loop through dataframe for city locaiton
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    cityName = row["City"]
    
    params["location"] = f"{lat},{lon}"
    
    response = requests.get(base_url, params=params).json()
    
    results = response ['results']
    
    # Collect name of first hotel
    # Set message when not enough information to collect and store hotel
    try:
        print(f"The closest hotel in {cityName} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = [results[0]["name"]]
    except(KeyError, IndexError):
        print("Not enough information; skipping.")
hotel_df

Not enough information; skipping.
Not enough information; skipping.
Not enough information; skipping.
Not enough information; skipping.
Not enough information; skipping.


,City,Country,Latitude,Longitude,Hotel
75,Jalu,LY,29.0331,21.5482,
225,Antalya Province,TR,36.7741,30.7178,
412,Port Said,EG,31.2565,32.2841,
462,Sweetwater,US,25.7634,-80.3731,
528,Athens,GR,37.9795,23.7162,


In [69]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotelInfo = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [70]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotelInfo)
fig.add_layer(markerLayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))